In [ ]:
%matplotlib inline

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import pytpc
import numpy as np
import matplotlib.pyplot as plt
from pytpc.constants import *
import seaborn as sns
import pandas as pd
from math import sqrt

In [ ]:
from scipy.interpolate import UnivariateSpline, LSQBivariateSpline
from scipy.integrate import quad

In [ ]:
sns.set_context('notebook', rc={'lines.markeredgewidth': 0.1})

In [ ]:
gas = pytpc.gases.InterpolatedGasMixture(150., ('helium', 0.9), ('carbon_dioxide', 0.1))
ef, bf = pytpc.utilities.create_fields(15e3, 0.5, 0)
pt = pytpc.Particle(4, 2, 1.3, position=(0, 0, 1.0), polar=pi - pi/4, azimuth=pi/4)
simres = pytpc.track(pt, gas, ef, bf)

In [ ]:
plt.plot(simres.x, simres.y)

In [ ]:
vd = pytpc.simulation.drift_velocity_vector(-3, 15e3, 0.5, 0)

In [ ]:
np.isscalar(vd)

In [ ]:
evt = pytpc.evtdata.make_event(simres[['x', 'y', 'z']].values*1e3, simres.de.values, 6.25, vd, 40., 4, 332, 100)

In [ ]:
plt.plot(evt.traces['data'].sum(0))

In [ ]:
meas = evt.xyzs(vd, 6.25, peaks_only=True)[:, :3]*1e-3
meas[:, 2] -= meas[:, 2].max() - 1.0

In [ ]:
plt.scatter(meas[:, 0], meas[:, 1], marker='o', c='goldenrod', edgecolor='none', s=10)
plt.plot(simres.x, simres.y)

In [ ]:
plt.plot(simres.z, simres.x)
plt.plot(simres.z, simres.y)
plt.plot(meas[:, 2], meas[:, 0], '.')
plt.plot(meas[:, 2], meas[:, 1], '.')

In [ ]:
%%timeit
sply = UnivariateSpline(meas[:, 2], meas[:, 1])
splx = UnivariateSpline(meas[:, 2], meas[:, 0])
dsply = sply.derivative()
dsplx = splx.derivative()

In [ ]:
%timeit [splx(z) for z in xs]
%timeit splx(xs)
%timeit list(map(splx, xs))

In [ ]:
xs = np.linspace(simres.z.min(), simres.z.max(), 100)
plt.plot(xs, splx(xs))
plt.plot(meas[:, 2], meas[:, 0], '.')
plt.plot(xs, sply(xs))
plt.plot(meas[:, 2], meas[:, 1], '.')
plt.plot(simres.z, simres.x, '--')
plt.plot(simres.z, simres.y, '--')

In [ ]:
fx = lambda z: sqrt(dsplx(z)**2 + dsply(z)**2 + 1)
%timeit quad(fx, 0.8, 0.801)

In [ ]:
print(quad(fx, 0.8, 0.81))
print(gx(0.8, 0.81))

In [ ]:
gx = lambda z1, z2: sqrt((splx(z2) - splx(z1))**2 + (sply(z2) - sply(z1))**2 + (z2-z1)**2)

In [ ]:
hx = lambda z1, z2: gx(z1, z2) - quad(fx, z1, z2)[0]

In [ ]:
hx(0.8, 0.9)

In [ ]:
np.sqrt(np.sum(np.diff(simres[['x', 'y', 'z']], axis=0)**2, axis=1)).sum()

In [ ]:
def arclen(a, npts):
    a = np.asanyarray(a)
    n = np.size(a, 0)
    idxs = np.linspace(0, n-1, npts, dtype='int')
    pts = a[idxs]
    diffs = np.diff(pts, axis=0)
    mags = np.linalg.norm(diffs, axis=-1)
    return mags.sum()
        

In [ ]:
alm = np.array([arclen(meas, n) for n in range(1,meas.shape[0])])
alp = np.array([arclen(pos, n) for n in range(1, pos.shape[0])])

In [ ]:
plt.plot(alm)
plt.plot(alp)
plt.xlim(0, 40)
plt.ylim(0, 1)

In [ ]:
plt.plot(alm-alp[-1])
plt.xlim(0, 20)
plt.ylim(-0.1, 0.1)

In [ ]:
d = np.linalg.norm(np.diff(pos, axis=0), axis=-1)

In [ ]:
arclen(meas, 5) - arclen(pos, pos.shape[0])

In [ ]:
arclen(meas, 4) / meas.shape[0]